# Start


Quick solution for tensorflow module not found


In [ ]:
!pip install tensorflow

In [ ]:
import os
import shutil

import kagglehub
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from PIL import Image

## Import data


In [ ]:
def import_raw_data():
    # Load dataset
    path = kagglehub.dataset_download("arashnic/faces-age-detection-dataset")
    
    # Define path to files
    faces_path  = os.path.join(path, "faces")
    train_path = os.path.join(faces_path, "Train")
    
    # Create directory for raw data
    export_path = "../data/raw/faces"
    os.makedirs(export_path, exist_ok=True)   
    
    # Copy files to raw data directory
    shutil.copytree(train_path, os.path.join(export_path, "images"), dirs_exist_ok=True)
    shutil.copy(os.path.join(faces_path, "train.csv"), os.path.join(export_path, "train.csv"))

    export_path = "../data/raw"
    

if (not os.path.exists("../data/raw/faces")):
    import_raw_data()

-   Faces_02 has no csv file for classification so we are gonna skip it


## Definitons


In [ ]:
df_train = pd.read_csv("../data/raw/faces/train.csv")
IMAGE_SIZE = 128

# 4.1


## A


-   EDA a data preprocessing pre Vami vybrané charakteristiky z datasetu


### EDA


#### Table content


In [ ]:
df_train.head()

In [ ]:
df_train.info()

In [ ]:
df_train.describe()

In [ ]:
df_train.duplicated().sum()

In [ ]:
df_train.isnull().sum()

In [ ]:
image_dir = "../data/raw/faces/images"
num_files = len([name for name in os.listdir(image_dir) if os.path.isfile(os.path.join(image_dir, name))])
print(f"Number of images: {num_files}")

In [ ]:
print(df_train["Class"].value_counts())
df_train["Class"].value_counts().plot(kind="bar")
plt.xticks(rotation=45)
plt.show()

-   There are no nulls
-   There are no duplicates
-   Number of images is same as number of data in cvs file


### Data preprocessing


#### Encoding


In [ ]:
class_mapping = {'YOUNG': 0, 'MIDDLE': 1, 'OLD': 2}
df_train["Class_enc"] = df_train["Class"].map(class_mapping)
df_train

#### Resizing images


In [ ]:
def resize_images(images, target_size):
    output_dir = "../data/resized/faces"
    os.makedirs(output_dir, exist_ok=True)
    for img_name in images["ID"]:
        img_path  = os.path.join("../data/raw/faces/images", img_name)

        img = Image.open(img_path)
        img_resized = img.resize(target_size)
        img_resized.save(os.path.join(output_dir, img_name))

if (not os.path.exists("../data/resized/faces")):
    resize_images(df_train, target_size=(IMAGE_SIZE, IMAGE_SIZE))

#### Splitting Data


In [ ]:
# df_test, df_train = train_test_split(df_combined, test_size=0.2, random_state=42)

## B


-   Zdôvodnite výber ML/DL metód vzhľadom na Vami vybraný dataset pre 4.2


# 4.2


## A


-   Modeluje Vami tie vybrané charakteristiky pomocou vhodných ML/DL
    metód. Výsledok modelovania je najlepší model.


## B


-   Zhodnotíte Váš prístup a získaný výsledok
